In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_csv('emoji_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [3]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [4]:
X = data[0].values
Y = data[1].values

# Embeddings

In [5]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [6]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [10]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

10


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [12]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [13]:
Ytrain = to_categorical(Y)

# Model

In [14]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [15]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [16]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
    
    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 4s 8ms/step - loss: 1.6068 - accuracy: 0.2295
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 1.5781 - accuracy: 0.3169
Epoch 3/100
6/6 [==============================] - 0s 8ms/step - loss: 1.5570 - accuracy: 0.3497
Epoch 4/100
6/6 [==============================] - 0s 8ms/step - loss: 1.5394 - accuracy: 0.3607
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 1.5223 - accuracy: 0.3443
Epoch 6/100
6/6 [==============================] - 0s 9ms/step - loss: 1.5042 - accuracy: 0.3497
Epoch 7/100
6/6 [==============================] - 0s 9ms/step - loss: 1.4831 - accuracy: 0.3880
Epoch 8/100
6/6 [==============================] - 0s 9ms/step - loss: 1.4612 - accuracy: 0.4481
Epoch 9/100
6/6 [==============================] - 0s 9ms/step - loss: 1.4338 - accuracy: 0.4754
Epoch 10/100
6/6 [==============================] - 0s 9ms/step - loss: 1.4019 - accuracy: 0.5246
Epoch 11/100
6/6 [===========

6/6 [==============================] - 0s 7ms/step - loss: 0.1996 - accuracy: 0.9727
Epoch 85/100
6/6 [==============================] - 0s 7ms/step - loss: 0.1932 - accuracy: 0.9727
Epoch 86/100
6/6 [==============================] - 0s 6ms/step - loss: 0.1889 - accuracy: 0.9727
Epoch 87/100
6/6 [==============================] - 0s 6ms/step - loss: 0.1852 - accuracy: 0.9727
Epoch 88/100
6/6 [==============================] - 0s 7ms/step - loss: 0.1861 - accuracy: 0.9727
Epoch 89/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2617 - accuracy: 0.9508
Epoch 90/100
6/6 [==============================] - 0s 7ms/step - loss: 0.2335 - accuracy: 0.9508
Epoch 91/100
6/6 [==============================] - 0s 7ms/step - loss: 0.1989 - accuracy: 0.9672
Epoch 92/100
6/6 [==============================] - 0s 6ms/step - loss: 0.1882 - accuracy: 0.9727
Epoch 93/100
6/6 [==============================] - 0s 7ms/step - loss: 0.1914 - accuracy: 0.9672
Epoch 94/100
6/6 [===============

In [19]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 [==============================] - 1s 759ms/step
I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
